In [1]:
%load_ext autoreload

%autoreload 2
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow.keras.callbacks as cb
import time
import numpy as np
from data import get_datasets
import matplotlib.pyplot as plt
import os
import shutil
from functools import partial
from data import get_datasets
from experiment import create_model, evaluate, find_adv_k, sentiment
import model_adv
from bertviz import head_view
import torch

In [2]:
%%javascript
require.config({
  paths: {
      d3: '//cdnjs.cloudflare.com/ajax/libs/d3/3.4.8/d3.min',
      jquery: '//ajax.googleapis.com/ajax/libs/jquery/2.0.0/jquery.min',
  }
});

<IPython.core.display.Javascript object>

In [3]:
(train_dataset, test_dataset), info = get_datasets(batch_size=64)
encoder = info.features['text'].encoder
vocab_size=info.features['text'].encoder.vocab_size 

In [4]:
transformer, _optimizer, _checkpoint, _manager = create_model(models_dir="models/pos_enc_True", load_checkpoint=True, 
                           vocab_size=vocab_size, use_positional_encoding=True, run_eagerly=True)

# adv = model_adv.create_model(models_dir="adv", load_checkpoint=False, run_eagerly=True, d_model=512)

Loaded previously trained model.


In [5]:
# Initial accuracy
# metrics = evaluate(transformer, test_dataset)
# print(metrics.result().numpy())

In [6]:
adv_model, adv_optimizer = model_adv.create_model(models_dir="adv", load_checkpoint=False, 
                                            run_eagerly=True, d_model=128)


In [9]:
x, y = next(iter(test_dataset))


sentence = x[0]
sentence_str = encoder.decode(x[0])
print(sentence_str)
tokens = [encoder.decode([w]) for w in sentence]

I've watched the movie actually several times. And what i want to say about it is the only thing that made this movie high rank was the Burak Altay's incredible performance, absolutely nothing but that. Not even those silly model named Deniz Akkaya and some of these popular names at times in the movie... Burak is definitely very talented i've seen a few jobs he made and been through. Even though this is kind of horror movie, he's doing really good job in comedy movies and also in dramas too. I bet most of you all saw Asmali Konak the movie and TV series, those two would go for an example... All i'm gonna say is you better watch out for the new works coming out from Burak then you'll see.. Keep the good work bro, much love..


In [39]:
#     for batch, (x, y) in enumerate(train_dataset):
loss_t, k, best_loss_t, best_adv_k = find_adv_k(x, y, transformer,
                adv_model, adv_optimizer, epochs=250)

Returned k: [  1   4 147 128]
[0] Loss_t = 0.008571237326      
[0] Loss_t (custom k) = 2.530784845         
[1 ] Loss_t = 3.503428459           Loss_k = -19.7434845            Loss = -8.120027542        
[2 ] Loss_t = 2.595817089           Loss_k = -19.76286697           Loss = -8.583524704        
[3 ] Loss_t = 1.033212543           Loss_k = -19.77846527           Loss = -9.372626305        
[4 ] Loss_t = 0.07953776419         Loss_k = -19.7920742            Loss = -9.856267929        
[5 ] Loss_t = 0.005254026968        Loss_k = -19.80577469           Loss = -9.900259972        
[6 ] Loss_t = 0.0008753265138       Loss_k = -19.81963921           Loss = -9.909381866        
[7 ] Loss_t = 0.0002890135802       Loss_k = -19.83363724           Loss = -9.91667366         
[8 ] Loss_t = 0.0001402318012       Loss_k = -19.84774208           Loss = -9.923800468        
[9 ] Loss_t = 8.392085147e-05       Loss_k = -19.86192513           Loss = -9.930920601        
[10] Loss_t = 5.730354678e-

[88] Loss_t = 7.900956916e-06       Loss_k = -20.96041679           Loss = -10.48020458        
[89] Loss_t = 7.898846889e-06       Loss_k = -20.97322083           Loss = -10.4866066         
[90] Loss_t = 7.897619071e-06       Loss_k = -20.98598289           Loss = -10.49298763        
[91] Loss_t = 7.896722309e-06       Loss_k = -20.99870682           Loss = -10.49934959        
[92] Loss_t = 7.896293937e-06       Loss_k = -21.01138878           Loss = -10.50569057        
[93] Loss_t = 7.896964235e-06       Loss_k = -21.02402878           Loss = -10.51201057        
[94] Loss_t = 7.89880869e-06        Loss_k = -21.03662872           Loss = -10.51831055        
[95] Loss_t = 7.900790479e-06       Loss_k = -21.04918671           Loss = -10.52458954        
[96] Loss_t = 7.903458936e-06       Loss_k = -21.06170464           Loss = -10.5308485         
[97] Loss_t = 7.906271094e-06       Loss_k = -21.0741806            Loss = -10.53708649        
[98] Loss_t = 7.909558917e-06       Loss

[173] Loss_t = 8.907660231e-06       Loss_k = -21.90626526           Loss = -10.95312786        
[174] Loss_t = 8.926019291e-06       Loss_k = -21.91583443           Loss = -10.95791245        
[175] Loss_t = 8.944485671e-06       Loss_k = -21.92537498           Loss = -10.96268272        
[176] Loss_t = 8.962887478e-06       Loss_k = -21.93488312           Loss = -10.96743679        
[177] Loss_t = 8.981574865e-06       Loss_k = -21.94436073           Loss = -10.9721756         
[178] Loss_t = 9.000360478e-06       Loss_k = -21.95380974           Loss = -10.9769001         
[179] Loss_t = 9.019092431e-06       Loss_k = -21.96322632           Loss = -10.98160839        
[180] Loss_t = 9.03772434e-06        Loss_k = -21.97261238           Loss = -10.98630142        
[181] Loss_t = 9.056525414e-06       Loss_k = -21.98196983           Loss = -10.99098015        
[182] Loss_t = 9.075364687e-06       Loss_k = -21.99129677           Loss = -10.99564362        
[183] Loss_t = 9.094269444e-06

In [40]:
# sentence = "This is a very terrible movie . I will never watch this bad movie again ."
# sentence = "this was a perfect movie . I enjoyed it a lot"
# tokens = sentence.split()

sent_original, _ = sentiment(sentence_str, encoder, transformer)
sent, weights = sentiment(sentence_str,    encoder=encoder, 
                      adv_model=adv_model, transformer=transformer)
# sent, weights = sentiment(sentence, encoder=encoder, transformer=old_transformer)


print(f"Initial sentiment: {sent_original}")
print(f'Adv sentiment: {sent}')

Initial sentiment: neg
Adv sentiment: pos


In [ ]:
head_view(torch.tensor(weights.numpy()[0]), tokens)

## Non Random K

In [ ]:
for batch, (x, y) in enumerate(train_dataset):
    loss_t, k, best_loss_t, best_adv_k = find_adv_k(x, y, transformer,
            adv_model, adv_optimizer, epochs=1, random_k=False)

In [8]:
# Adv after 1 epochs of training
metrics = evaluate(transformer, test_dataset, adv_model=adv_model)
print(metrics.result().numpy())

0.836199


In [ ]:
for batch, (x, y) in enumerate(train_dataset):
    loss_t, k, best_loss_t, best_adv_k = find_adv_k(x, y, transformer,
            adv_model, adv_optimizer, epochs=1, random_k=False)

In [11]:
# Adv after 2 epochs of training
metrics = evaluate(transformer, test_dataset, adv_model=adv_model)
print(metrics.result().numpy())

0.8323017


In [12]:
# Original accuracy
metrics = evaluate(transformer, test_dataset)
print(metrics.result().numpy())

0.82691425


## Random K

In [7]:
# Original accuracy
metrics = evaluate(transformer, test_dataset)
print(metrics.result().numpy())

0.82691425


In [ ]:
for batch, (x, y) in enumerate(train_dataset):
    loss_t, k, best_loss_t, best_adv_k = find_adv_k(x, y, transformer,
            adv_model, adv_optimizer, epochs=1, random_k=True)

In [9]:
# Adv after 1 epochs of training
metrics = evaluate(transformer, test_dataset, adv_model=adv_model)
print(metrics.result().numpy())

0.5600642


In [ ]:
for batch, (x, y) in enumerate(train_dataset):
    loss_t, k, best_loss_t, best_adv_k = find_adv_k(x, y, transformer,
            adv_model, adv_optimizer, epochs=1, random_k=True)

In [11]:
# Adv after 2 epochs of training
metrics = evaluate(transformer, test_dataset, adv_model=adv_model)
print(metrics.result().numpy())

0.5712976


In [12]:
for batch, (x, y) in enumerate(train_dataset):
    loss_t, k, best_loss_t, best_adv_k = find_adv_k(x, y, transformer,
            adv_model, adv_optimizer, epochs=1, random_k=True)

Returned k: [ 64   4 199 128]
[0] Loss_t = 0.05019379407       
[0] Loss_t (custom k) = 0.4156490266        
[1 ] Loss_t = 1.12534833            Loss_k = -21.55042267           Loss = -10.21253681        
Returned k: [ 64   4 199 128]
[0] Loss_t = 0.08819077164       
[0] Loss_t (custom k) = 0.3886458576        
[1 ] Loss_t = 0.8949867487          Loss_k = -21.55884743           Loss = -10.33193016        
Returned k: [ 64   4 199 128]
[0] Loss_t = 0.1419844031        
[0] Loss_t (custom k) = 0.5051864982        
[1 ] Loss_t = 0.6403122544          Loss_k = -21.56709099           Loss = -10.4633894         
Returned k: [ 64   4 193 128]
[0] Loss_t = 0.2370337248        
[0] Loss_t (custom k) = 0.8570222259        
[1 ] Loss_t = 1.573281288           Loss_k = -21.5439415            Loss = -9.985330582        
Returned k: [ 64   4 199 128]
[0] Loss_t = 0.2232460678        
[0] Loss_t (custom k) = 0.5425593853        
[1 ] Loss_t = 1.148141146           Loss_k = -21.58238411           Los

Returned k: [ 64   4 199 128]
[0] Loss_t = 0.1824121773        
[0] Loss_t (custom k) = 0.490432471         
[1 ] Loss_t = 1.247738719           Loss_k = -21.83085632           Loss = -10.29155922        
Returned k: [ 64   4 199 128]
[0] Loss_t = 0.154166311         
[0] Loss_t (custom k) = 0.5944890976        
[1 ] Loss_t = 1.473010302           Loss_k = -21.83757782           Loss = -10.1822834         
Returned k: [ 64   4 200 128]
[0] Loss_t = 0.1150390357        
[0] Loss_t (custom k) = 0.3829343319        
[1 ] Loss_t = 0.9532404542          Loss_k = -21.84903526           Loss = -10.44789696        
Returned k: [ 64   4 200 128]
[0] Loss_t = 0.2117867172        
[0] Loss_t (custom k) = 0.7234801054        
[1 ] Loss_t = 2.132094383           Loss_k = -21.85451889           Loss = -9.861211777        
Returned k: [ 64   4 199 128]
[0] Loss_t = 0.147298187         
[0] Loss_t (custom k) = 0.6491444707        
[1 ] Loss_t = 1.432781696           Loss_k = -21.85672951           Los

Returned k: [ 64   4 200 128]
[0] Loss_t = 0.1936954558        
[0] Loss_t (custom k) = 0.6994796991        
[1 ] Loss_t = 1.90695703            Loss_k = -22.08358765           Loss = -10.08831501        
Returned k: [ 64   4 197 128]
[0] Loss_t = 0.1181766689        
[0] Loss_t (custom k) = 0.6271260977        
[1 ] Loss_t = 0.9521891475          Loss_k = -22.074049             Loss = -10.56093025        
Returned k: [ 64   4 200 128]
[0] Loss_t = 0.1755669564        
[0] Loss_t (custom k) = 0.5598931313        
[1 ] Loss_t = 1.231748104           Loss_k = -22.09534264           Loss = -10.43179703        
Returned k: [ 64   4 199 128]
[0] Loss_t = 0.1320533007        
[0] Loss_t (custom k) = 0.5918931961        
[1 ] Loss_t = 1.522198439           Loss_k = -22.09608078           Loss = -10.28694153        
Returned k: [ 64   4 197 128]
[0] Loss_t = 0.05797974765       
[0] Loss_t (custom k) = 0.5313856602        
[1 ] Loss_t = 1.371095181           Loss_k = -22.09152985           Los

Returned k: [ 64   4 200 128]
[0] Loss_t = 0.1116945669        
[0] Loss_t (custom k) = 0.4635518491        
[1 ] Loss_t = 1.594461679           Loss_k = -22.30764961           Loss = -10.35659409        
Returned k: [ 64   4 200 128]
[0] Loss_t = 0.1613087058        
[0] Loss_t (custom k) = 0.4558310211        
[1 ] Loss_t = 0.6658443213          Loss_k = -22.31364441           Loss = -10.82390022        
Returned k: [ 64   4 200 128]
[0] Loss_t = 0.1760733426        
[0] Loss_t (custom k) = 0.6082619429        
[1 ] Loss_t = 1.020107985           Loss_k = -22.31825256           Loss = -10.64907265        
Returned k: [ 64   4 200 128]
[0] Loss_t = 0.1917080879        
[0] Loss_t (custom k) = 0.6535028815        
[1 ] Loss_t = 1.752773762           Loss_k = -22.32389641           Loss = -10.28556156        
Returned k: [ 64   4 200 128]
[0] Loss_t = 0.1610500216        
[0] Loss_t (custom k) = 0.5577352047        
[1 ] Loss_t = 0.6204389334          Loss_k = -22.3295784            Los

In [13]:
# Adv after 3 epochs of training
metrics = evaluate(transformer, test_dataset, adv_model=adv_model)
print(metrics.result().numpy())

0.56591016


In [ ]:
# Train with random K for longer

history = []

for e in range(10):
    print(f'Epoch {e+1}')
    for batch, (x, y) in enumerate(train_dataset):
        loss_t, k, best_loss_t, best_adv_k = find_adv_k(x, y, transformer,
                adv_model, adv_optimizer, epochs=50, random_k=True)
        
    metrics = evaluate(transformer, test_dataset, adv_model=adv_model)
    print(metrics.result().numpy())
    history.append(metrics)

# Final accuracy:
0.6944062

In [28]:
def visualize():
    x, y = next(iter(train_dataset))

    sentence = x[0].numpy().tolist()
    sentence = sentence[0:sentence.index(0)]
    print(sentence)
    sentence_str = encoder.decode(x[0])    
    print(sentence_str)
    tokens = [encoder.decode([w]) for w in sentence]
    
    print(len(tokens))
    sent_original, _ = sentiment(sentence_str, encoder, transformer)
    sent, weights = sentiment(sentence_str,    encoder=encoder, 
                      adv_model=adv_model, transformer=transformer)
    # sent, weights = sentiment(sentence, encoder=encoder, transformer=old_transformer)

    print(f"Initial sentiment: {sent_original}")
    print(f'Adv sentiment: {sent}')
    head_view(torch.tensor(weights.numpy()[0]), tokens)

In [ ]:
visualize()